In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import requests

In [2]:
df = pd.read_parquet('./data/carpark_history_6_months.parquet')
df.head()

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,207210,741448862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:01:02,6,Gordon Henry St North Car Park,207210TPR001
1,207210,741450092,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:21:32,6,Gordon Henry St North Car Park,207210TPR001
2,207210,741468991,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:36:31,6,Gordon Henry St North Car Park,207210TPR001
3,207210,741469862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:51:02,6,Gordon Henry St North Car Park,207210TPR001
4,207210,741474196,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T07:03:16,6,Gordon Henry St North Car Park,207210TPR001


In [3]:
carparks_df_copy=df.copy()

In [4]:
def extract_date_time(message_date):
  date = message_date.split('T')[0]
  time = message_date.split('T')[1]
  return date,time

# Separate the date and time in to 2 separate columns
carparks_df_copy[['date','time']] = df['MessageDate'].apply(extract_date_time).apply(pd.Series)
carparks_df_copy.head()

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id,date
0,207210,00:01:02,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:01:02,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01
1,207210,00:21:32,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:21:32,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01
2,207210,05:36:31,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:36:31,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01
3,207210,05:51:02,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:51:02,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01
4,207210,07:03:16,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T07:03:16,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01


In [5]:
# Indicate the day of the week according to the date
carparks_df_copy['day_of_week'] = pd.to_datetime(carparks_df_copy['date']).apply(lambda x: x.strftime('%A'))
carparks_df_copy.head()

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id,date,day_of_week
0,207210,00:01:02,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:01:02,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01,Saturday
1,207210,00:21:32,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:21:32,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01,Saturday
2,207210,05:36:31,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:36:31,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01,Saturday
3,207210,05:51:02,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:51:02,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01,Saturday
4,207210,07:03:16,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T07:03:16,6,Gordon Henry St North Car Park,207210TPR001,2023-07-01,Saturday


In [6]:
# Drop unnecessary columns
carparks_df_copy.drop(['tsn', 'occupancy','tfnsw_facility_id', 'MessageDate'], axis=1, inplace=True)
carparks_df_copy.head()

,time,spots,zones,ParkID,facility_id,facility_name,date,day_of_week
0,00:01:02,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,6,Gordon Henry St North Car Park,2023-07-01,Saturday
1,00:21:32,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,6,Gordon Henry St North Car Park,2023-07-01,Saturday
2,05:36:31,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,6,Gordon Henry St North Car Park,2023-07-01,Saturday
3,05:51:02,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,6,Gordon Henry St North Car Park,2023-07-01,Saturday
4,07:03:16,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,6,Gordon Henry St North Car Park,2023-07-01,Saturday
